<a href="https://colab.research.google.com/github/karandomguy/iriscnn-dailymailsummariser/blob/tests/cnn_dailymailtextsummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd

## Installing dataset from Kaggle

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail


newspaper-text-summarization-cnn-dailymail.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip newspaper-text-summarization-cnn-dailymail.zip

Archive:  newspaper-text-summarization-cnn-dailymail.zip
replace cnn_dailymail/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace cnn_dailymail/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace cnn_dailymail/validation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
train_df = pd.read_csv('cnn_dailymail/train.csv')
train_df

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
287108,fffdfb56fdf1a12d364562cc2b9b1d4de7481dee,By . James Rush . Former first daughter Chelse...,Chelsea Clinton said question of running for o...
287109,fffeecb8690b85de8c3faed80adbc7a978f9ae2a,An apologetic Vanilla Ice has given his first ...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,ffff5231e4c71544bc6c97015cdb16c60e42b3f4,America's most lethal sniper claimed he wished...,America's most lethal sniper made comment in i...
287111,ffff924b14a8d82058b6c1c5368ff1113c1632af,"By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar...",A swarm of more than one million has crossed b...


In [ ]:
test_data = pd.read_csv('cnn_dailymail/test.csv')

In [ ]:
train_df = train_df.drop(['id'],axis=1) # dropping id column

In [ ]:
train_data=train_df

In [ ]:
train_data = train_data.reset_index(drop=True)
test_data = test_data.drop(['id'], axis=1)
test_data = test_data.reset_index(drop=True)

# **Extractive Summarization using BERT**



In [ ]:
from summarizer import Summarizer
from pprint import pprint

In [ ]:
!pip install transformers
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
train_check=train_data[1:5]

In [ ]:
train_check

,article,highlights
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
from transformers import pipelines

In [ ]:

!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Example 1


In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
ARTICLE=train_check['article'][2]
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [ ]:
max_chunk = 500
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [ ]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [ ]:
output=str(res[0])

In [ ]:
train_check['highlights'][2]

"Craig Eccleston-Todd, 27, had drunk at least three pints before driving car .\nWas using phone when he veered across road in Yarmouth, Isle of Wight .\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital .\nPolice say he would have been over legal drink-drive limit at time of crash .\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving ."

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(output,
                      train_check['highlights'][2])

In [ ]:
scores

{'rouge1': Score(precision=0.5142857142857142, recall=0.5, fmeasure=0.5070422535211268),
 'rougeL': Score(precision=0.2857142857142857, recall=0.2777777777777778, fmeasure=0.28169014084507044)}

Example 2

In [ ]:
ARTICLE2=train_check['article'][3]
ARTICLE2 = ARTICLE2.replace('.', '.<eos>')
ARTICLE2 = ARTICLE2.replace('?', '?<eos>')
ARTICLE2 = ARTICLE2.replace('!', '!<eos>')

In [ ]:
max_chunk = 500
sentences = ARTICLE2.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [ ]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

Your max_length is set to 120, but you input_length is only 68. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)


In [ ]:
output2=str(res[0])

In [ ]:
train_check['highlights'][3]

"Nina dos Santos says Europe must be ready to accept sanctions will hurt both sides .\nTargeting Russia's business community would be one way of sapping their support for President Putin, she says .\nBut she says Europe would have a hard time keeping its factories going without power from the east ."

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(output2,
                      train_check['highlights'][3])

In [ ]:
scores

{'rouge1': Score(precision=0.3333333333333333, recall=0.25757575757575757, fmeasure=0.2905982905982906),
 'rougeL': Score(precision=0.17647058823529413, recall=0.13636363636363635, fmeasure=0.15384615384615383)}

Example 3

In [ ]:
ARTICLE3=train_check['article'][4]
ARTICLE3 = ARTICLE3.replace('.', '.<eos>')
ARTICLE3 = ARTICLE3.replace('?', '?<eos>')
ARTICLE3 = ARTICLE3.replace('!', '!<eos>')

In [ ]:
max_chunk = 500
sentences = ARTICLE3.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [ ]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [ ]:
output3=str(res[0])

In [ ]:
train_check['highlights'][4]

'Fleetwood top of League One after 2-0 win at Scunthorpe .\nPeterborough, Bristol City, Chesterfield and Crawley all drop first points of the season .\nStand-in striker Matt Done scores a hat-trick as Rochdale thrash Crewe 5-2 .\nWins for Notts County and Yeovil .\nCoventry/Bradford and Oldham/Port Vale both end in draws .\nA late Stephen Bywater own goal denies Gillingham three points against Millwall .'

In [ ]:
output3

"{'summary_text': ' fleetwood are the only team still to have a 100% record in sky bet league one . graham alexander’s men beat scunthorpe 2-0 to go top of the table . posh were defeated 2-1 by sheffield united, who had lost both of their opening games . bristol city were held to a goalless draw by leyton orient . crawley lost their unbeaten status, while bradford maintained theirs thanks to a 3-1 win for the bantams .'}"

In [ ]:
train_check['article'][4]

"fleetwood are the only team still to have a 100% record in sky bet league one as a 2-0 win over scunthorpe sent graham alexander’s men top of the table. the cod army are playing in the third tier for the first time in their history after six promotions in nine years and their remarkable ascent shows no sign of slowing with jamie proctor and gareth evans scoring the goals at glanford park. fleetwood were one of five teams to have won two out of two but the other four clubs - peterborough, bristol city, chesterfield and crawley - all hit their first stumbling blocks. posh were defeated 2-1 by sheffield united, who had lost both of their opening contests. jose baxter’s opener gave the blades a first-half lead, and although it was later cancelled out by shaun brisley’s goal, ben davies snatched a winner six minutes from time. in the lead: jose baxter (right) celebrates opening the scoring for sheffield united . up for the battle: sheffield united's michael doyle (left) challenges peterbor

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(output3,
                      train_check['highlights'][4])

In [ ]:
scores

{'rouge1': Score(precision=0.3283582089552239, recall=0.2857142857142857, fmeasure=0.3055555555555556),
 'rougeL': Score(precision=0.16417910447761194, recall=0.14285714285714285, fmeasure=0.1527777777777778)}

Example 4



In [ ]:
ARTICLE4=train_data['article'][5]
ARTICLE4 = ARTICLE4.replace('.', '.<eos>')
ARTICLE4 = ARTICLE4.replace('?', '?<eos>')
ARTICLE4 = ARTICLE4.replace('!', '!<eos>')

In [ ]:
max_chunk = 500
sentences = ARTICLE4.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [ ]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [ ]:
output4=str(res[0])

In [ ]:
train_data['highlights'][5]

'Prime Minister and his family are enjoying an Easter break in Lanzarote .\nSported the same £20.99 beige loafers as he wore in Portugal last year .\nPM sat and had a drink at a beach-side cafe on the Spanish Island .'

In [ ]:
output4

"{'summary_text': ' David Cameron was seen in the same pair of beige loafers he wore on holiday last year . He got the £20.99 shoes from high street store Aldo and took them with him to Portugal last summer . He teamed them with a navy blue shirt and beige shorts on a trip to Teguise in the centre of the island .'}"

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(output4,
                      train_data['highlights'][5])

In [ ]:
scores

{'rouge1': Score(precision=0.4878048780487805, recall=0.32786885245901637, fmeasure=0.39215686274509803),
 'rougeL': Score(precision=0.34146341463414637, recall=0.22950819672131148, fmeasure=0.27450980392156865)}

Example 5

In [ ]:
ARTICLE5=train_data['article'][6]
ARTICLE5 = ARTICLE5.replace('.', '.<eos>')
ARTICLE5 = ARTICLE5.replace('?', '?<eos>')
ARTICLE5 = ARTICLE5.replace('!', '!<eos>')

In [ ]:
max_chunk = 500
sentences = ARTICLE5.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [ ]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [ ]:
output5=str(res[0])

In [ ]:
train_data['highlights'][6]

'NBA star calls for black and Hispanic communities to get tested .\nFormer Lakers player dedicated life to raising awareness about disease .'

In [ ]:
output5

'{\'summary_text\': " Magic Johnson was \'devastated\' when he was told he had HIV in 1991 . The former Los Angeles Lakers player is now campaigning for more people to get tested for the disease ."}'

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(output5,
                      train_data['highlights'][6])

In [ ]:
scores

{'rouge1': Score(precision=0.38095238095238093, recall=0.24242424242424243, fmeasure=0.2962962962962963),
 'rougeL': Score(precision=0.23809523809523808, recall=0.15151515151515152, fmeasure=0.18518518518518517)}